In [2]:
import h2o
import os
from h2o.estimators import H2ORandomForestEstimator
h2o.init()

# Define the path to the file
file_path = "C:/Users/Utilizador/Desktop/Cenas_do_Instituto/Ano3/IA/IA2324SlaughterSquad/SlaughterSquad/src/main/java/com/slaughtersquad/datasets/dataset.csv"

# Import the file
raw_data = h2o.import_file(path = [file_path], parse = False)

parse_setup = h2o.parse_setup(
    raw_frames = raw_data, 
    destination_frame = "parsed_data", 
    separator = ";",
    column_names = ["currentPositionX", "currentPositionY", "distance", "velocity", "bearing", "futureBearing", "enemyPositionX", "enemyPositionY", "predictedEnemyPositionX", "predictedEnemyPositionY", "gunTurnRemaining", "gunHeat", "hitOrNot"],
    column_types = ["numeric", "numeric", "numeric", "numeric", "numeric", "numeric", "numeric", "numeric", "numeric", "numeric", "numeric", "numeric", "enum"],
    header = 1,
)

# Parse the file with the specified setup options
df = h2o.parse_raw(parse_setup)

# Display the dataframe
df.head()

# Define features (or predictors) and target (or response)
features = [
    "currentPositionX", "currentPositionY", "distance", 
    "velocity", "bearing", "futureBearing", 
    "enemyPositionX", "enemyPositionY", 
    "predictedEnemyPositionX", "predictedEnemyPositionY", 
    "gunTurnRemaining", "gunHeat"
]

target = "hitOrNot" # Binary target column

# Ensure the target is binary (factor)
df[target] = df[target].asfactor()

# Split the data into training, validation, and test sets
train, valid, test = df.split_frame(
    ratios = [0.7, 0.15],
    seed = 123
)

# Train the model
model = H2ORandomForestEstimator(ntrees = 100, max_depth = 20, min_rows = 10)
model.train(x = features, y = target, training_frame = train, validation_frame = valid)

model_type = "RandomForest"
file_path = "C:/Users/Utilizador/Desktop/Cenas_do_Instituto/Ano3/IA/IA2324SlaughterSquad/SlaughterSquad/target/classes/com/slaughtersquad/sampleRobots/IntelligentRobot.data"

# Save the model
mojo_path = model.download_mojo(path = file_path, get_genmodel_jar = True)

# Get the model type --> Regression, Binomial, Multinomial, Clustering, AutoEncoder, DimReduction, WordEmbedding
model_category = model.type

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,3 mins 49 secs
H2O_cluster_timezone:,Europe/Lisbon
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,2 months and 14 days
H2O_cluster_name:,H2O_from_python_Utilizador_izb3ui
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.886 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


ValueError: length of col_names should be equal to the number of columns: 12 vs 13